In [1]:
require 'nn'
require 'itorch'
require 'image'
require 'optim'
require 'xlua'

require 'model.VanillaLSTM'
require 'util.DataLoader'
require 'util.BatchLoader'
require 'util.SlidingWindowDataLoader'
require 'util.KFoldSlidingWindowDataLoader'
require 'util.DenseSlidingWindowDataLoader'
require 'model.ConvLSTM'
require 'model.DenseConvLSTM'

utils = require 'util.utils'

dtype = 'torch.FloatTensor'


In [2]:
opt = {}
-- Dataset options
opt.input_h5 = '../../openface_data/mohit_data.h5'
opt.train_seq_h5 = '../../openface_data/main_gest_by_file_1.h5'
opt.data_dir = '../../openface_data/face_gestures/dataseto_text'
opt.aug_gests_h5 = '../../openface_data/main_gest_by_file_aug_1_K_32.h5'
opt.batch_size = 200
opt.num_classes = 11
opt.num_classify = 5
opt.win_len = 10
opt.win_step = 5
opt.num_features = 52
-- Return the list of which batch inputs are in the current batch for validation
-- data
opt.val_batch_info = 1

-- Model options
opt.init_from = './3_win_conv_lstm_aug_2_lr_0001_win_step_3/checkpoint_60.t7'
opt.use_dense_conv_lstm = 0

-- Output options
--[[
cmd:option('-save', 'dense_step_5_cls_5')
cmd:option('-checkpoint_name', 'checkpoint')
cmd:option('-test_log', 'test.log')
cmd:option('-test_batch', 'test_batch.json')
cmd:option('-test_scores', 'test_scores.json')
cmd:option('-test_preds', 'test_preds.json')
]]

-- Backend options
opt.gpu = 1
opt.gpu_backend = 'cuda'

--[[
opt.test_log= opt.save..'/'..opt.test_log
opt.test_scores = opt.save .. '/' .. opt.test_scores
opt.test_preds = opt.save .. '/' .. opt.test_preds
]]

--[[
local trainLogger = optim.Logger(paths.concat(opt.save, opt.train_log))
if opt.use_dense_conv_lstm == 1 then
  trainLogger:setNames{'train_err', 'train_loss',}
else
  trainLogger:setNames{'train_err', 'train_loss', 'l_inf_w1_16',
    'l_inf_w1_32', 'l_inf_w1_64'}
end
]]

In [3]:
if opt.gpu == 1 then
  require 'cutorch'
  require 'cunn'
  require 'cudnn'
  cutorch.setDevice(1)
  dtype = 'torch.CudaTensor'
  print(string.format('Running with CUDA on GPU 1'))
else
  print('Running in CPU mode')
end

-- Confusion matrix
local classes = {}
for i=1,opt.num_classify do table.insert(classes, i) end
-- Confusion matrix for data
conf = optim.ConfusionMatrix(classes)

opt_clone = torch.deserialize(torch.serialize(opt))

print('Will load model '..opt.init_from)
-- Load model
model = torch.load(opt_clone.init_from)

print('Did load model')

print(model.conf)
model = model.model
--print(model)

Running with CUDA on GPU 1	
Will load model ./3_win_conv_lstm_aug_2_lr_0001_win_step_3/checkpoint_60.t7	


In [4]:
function getSampleTestSignal()
    local gest_by_type = utils.get_all_gest_by_type(
        opt.train_seq_h5, 'test', 11)
    local inp = utils.get_input_from_gest_type(gest_by_type, 11, 16)
    return inp
end
sample_inp = getSampleTestSignal()

Did load model	
ConfusionMatrix:
[[   29069    2865     332     241     606]   87.787% 	[class: 1]
 [    4019    3170      10       7     157]   43.053% 	[class: 2]
 [    1474     121     194       6      55]   10.486% 	[class: 3]
 [    1376     153      12     209      85]   11.390% 	[class: 4]
 [     855     269       0       0     115]]  9.282% 	[class: 5]
 + average row correct: 32.39963799715% 
 + average rowUcol correct (VOC measure): 24.957930669188% 
 + global correct: 72.151982378855%	


In [5]:
local w1_16, grad_w1_16 = model.net:get(1):get(1):get(1):parameters()
-- Get the weights not the biases
w1_16 = w1_16[1]
itorch.image(w1_16)

local rand_inp = sample_inp[1][1]
rand_inp = rand_inp:view(1, 16, 53)
local new_conv_net = nn.SpatialConvolution(1,128,1,3)
new_conv_net.weights = w1_16
local final_op = new_conv_net:forward(rand_inp)

Plot = require 'itorch.Plot'

local nod1 = sample_inp[1][1]
local nod1_nosetip_x = nod1[{{}, {18}}]
nod1_nosetip_x = nod1_nosetip_x:reshape(16)
local nod1_nosetip_y = nod1[{{}, {19}}]:reshape(16)
p1 = Plot():line(torch.linspace(0, 15, 16), nod1_nosetip_x, 'red', 'nod i/p nosetip x'):legend(true):draw()
p2 = Plot():line(torch.linspace(0, 15, 16), nod1_nosetip_y, 'red', 'nod i/p nosetip y'):legend(true):draw()

local nod1_op_nosetip_x = final_op[{{1}, {}, {18}}]:reshape(14)
local nod1_op_nosetip_y = final_op[{{1}, {}, {19}}]:reshape(14)
p3 = Plot():line(torch.linspace(0, 13, 14), nod1_op_nosetip_x, 'blue', 'nod o/p nosetip x'):legend(true):draw()
p4 = Plot():line(torch.linspace(0, 13, 14), nod1_op_nosetip_y, 'blue', 'nod o/p nosetip y'):legend(true):draw()


 128
  14
  53
[torch.LongStorage of size 3]



Columns 1 to 6
-4.2415e-01  2.6733e-01 -8.6524e-01 -5.6692e-01  2.3993e-01 -7.8806e-01
-4.2952e-01  2.6712e-01 -8.6264e-01 -5.7110e-01  2.4265e-01 -7.8420e-01
-4.3185e-01  2.6289e-01 -8.6278e-01 -5.7370e-01  2.3864e-01 -7.8353e-01
-4.3088e-01  2.5292e-01 -8.6624e-01 -5.7233e-01  2.2834e-01 -7.8759e-01
-4.3567e-01  2.5302e-01 -8.6381e-01 -5.8326e-01  2.1949e-01 -7.8207e-01
-4.3784e-01  2.4959e-01 -8.6372e-01 -5.9254e-01  2.0858e-01 -7.7807e-01
-4.4088e-01  2.4571e-01 -8.6328e-01 -5.9857e-01  2.0212e-01 -7.7515e-01
 1.4543e-01  1.3274e-02 -9.8928e-01 -2.1901e-02  2.7615e-03 -9.9976e-01
 1.4678e-01  1.7798e-02 -9.8901e-01 -1.9586e-02  5.2313e-03 -9.9979e-01
 1.4568e-01  1.8877e-02 -9.8915e-01 -2.1676e-02  6.4055e-03 -9.9974e-01
 1.4218e-01  2.4533e-02 -9.8954e-01 -2.5291e-02  1.4469e-02 -9.9957e-01
 1.4410e-01  2.9099e-02 -9.8914e-01 -2.3273e-02  1.9335e-02 -9.9954e-01
 1.4578e-01  3.3727e-02 -9.8874e-01 -2.0228e-02  2.3702e-02 -9.9951e-01
 1.4649e-01  3.4626e-02 -9.8860e-01 -1.6476e-02  

+00
-1.9714e+00  2.7100e+00 -2.1473e+00  2.6676e+00 -2.1556e+00  2.3036e+00
-1.9963e+00  2.9677e+00 -2.1818e+00  2.9359e+00 -2.1966e+00  2.5506e+00
-2.3149e+00  3.3547e+00 -2.2106e+00  3.3887e+00 -2.2255e+00  3.0191e+00
-2.2864e+00  3.3901e+00 -2.1849e+00  3.4405e+00 -2.1854e+00  3.1309e+00
-2.1870e+00  3.3227e+00 -2.1541e+00  3.3466e+00 -2.1247e+00  3.0694e+00
-2.1009e+00  3.1228e+00 -2.0614e+00  3.1355e+00 -2.0210e+00  2.9324e+00
-1.9829e+00  2.7794e+00 -1.9427e+00  2.8018e+00 -1.8720e+00  2.7157e+00
-1.7409e+00  1.9613e+00 -1.6673e+00  2.0177e+00 -1.5323e+00  2.1405e+00

Columns 25 to 30
-1.9364e+01  9.0835e+01 -1.9046e+01  9.5755e+01 -1.8283e+01  9.1003e+01
-1.8928e+01  9.1222e+01 -1.8643e+01  9.6142e+01 -1.7957e+01  9.1376e+01
-1.8547e+01  9.1655e+01 -1.8277e+01  9.6550e+01 -1.7607e+01  9.1733e+01
-1.8246e+01  9.2144e+01 -1.7990e+01  9.6936e+01 -1.7254e+01  9.2074e+01
-1.8069e+01  9.2640e+01 -1.7681e+01  9.7716e+01 -1.6615e+01  9.2978e+01
-1.8087e+01  9.3292e+01 -1.7627e+01  9.847

ns 43 to 48
-6.6690e+00 -3.3200e+00 -4.0840e+00 -3.8400e+00 -3.3260e+00  4.7100e+00
-6.2090e+00 -3.3600e+00 -5.2050e+00 -3.4200e+00 -5.6400e+00  4.8900e+00
-5.6270e+00 -3.5500e+00 -5.5780e+00 -3.2800e+00 -6.3670e+00  5.0300e+00
-4.4240e+00 -3.3500e+00 -5.6320e+00 -3.1400e+00 -6.1540e+00  5.0300e+00
-4.7470e+00 -6.7800e+00 -1.0191e+01 -8.3000e+00 -9.0450e+00  6.0500e+00
-8.4100e-01 -6.5600e+00 -4.3440e+00 -7.2500e+00 -5.0550e+00  6.7500e+00
 1.9300e-01 -5.7500e+00 -2.0760e+00 -6.7600e+00 -1.3960e+00  6.9400e+00
-2.4333e+02  8.4205e+02 -2.3396e+02  8.4818e+02 -2.3373e+02 -1.1600e-01
 7.9600e-01 -2.5000e+00  1.8130e+00 -2.1990e+00  1.0000e+00 -1.1000e-02
 4.9200e-01 -2.3070e+00  7.3100e-01 -2.3640e+00  5.9700e-01  1.0300e-01
 1.3700e+00 -3.2220e+00 -1.9800e-01 -2.7480e+00  4.8000e-02 -7.9100e-01
-2.1100e-01 -4.4700e-01  6.8000e-02 -1.6600e-01 -4.5000e-02 -9.7000e-01
-1.1750e+00  5.5800e-01 -3.1500e-01  5.3600e-01 -1.9500e-01 -7.0700e-01
-1.6280e+00  1.2040e+00 -8.1700e-01  1.4880e+00 -5.3

In [ ]:
model = model:cuda()
local softmaxLayer = cudnn.SoftMax():cuda()
model.net:add(softmaxLayer)

In [ ]:
-- Load data

local loader 
local val_data_co
if opt.use_dense_conv_lstm == 1 then
  loader = DenseSlidingWindowDataLoader(opt_clone)
  val_data_co = coroutine.create(DenseSlidingWindowDataLoader.next_val_batch)
else
  loader = KFoldSlidingWindowDataLoader(opt_clone)
  val_data_co = coroutine.create(KFoldSlidingWindowDataLoader.next_val_batch)
end

In [ ]:
-- Evaluate

conf:zero()
local num_test = 0
local test_scores = {}
local test_preds = {}
local test_data = {}

-- Evaluate model
model:evaluate()

while coroutine.status(val_data_co) ~= 'dead' do
  local success, xv, yv, batch = coroutine.resume(val_data_co, loader) 
  if success and xv ~= nil then
    for i=1,#xv do xv[i] = xv[i]:type(dtype) end
    yv = yv:type(dtype)
    local scores = model:forward(xv)
    local scores_max, scores_max_idx = torch.max(scores, 2)
    conf:batchAdd(scores, yv)
    num_test = num_test + 1
    -- Add batch data to the global table
    for i=1,#batch do table.insert(test_data, batch[i]) end
    -- Add the predictions to the global table
    scores_max_idx = torch.totable(scores_max_idx)
    for i=1,#scores_max_idx do table.insert(test_preds, scores_max_idx[i]) end
    -- Add the scores to the global table
    scores = torch.totable(scores)
    for i=1, #scores do table.insert(test_scores, scores[i]) end
  elseif success ~= true then
    print('Validation data coroutine failed')
    print(xv)
  end
end

print('Total frames evaluated '..num_test)
print(conf)